# Models

> Models Available For This Project

In [ ]:
#| default_exp modeling.models

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
from data_system_utilities.file_parsers import yaml

In [ ]:
#| export


from machine_learning_utilities.training.hypertuning import * # noqa:
from sklearn import metrics
from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, Trials, space_eval
from sklearn.linear_model import LogisticRegression
from rfpimp import * # noqa:

import xgboost as xgb
import numpy as np
import logging


logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.core").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

This isn't in MSU yet so I am adding it here

In [ ]:
#| export




class HpOptBinary:
    """Class that hypertunes an arbitrary model for binary classification"""

    def __init__(
        self,
        X_train,
        X_test,
        y_train,
        y_test,
        parameter_space=None,
        model=xgb.XGBClassifier,
        loss_function=metrics.roc_auc_score,
        loss_params={},
        loss_pred_proba=True,
    ):
        """Initialization takes in a test and train set and an optional hyperparameter space

        Args:
            X_train (array): training features
            X_test (array): testing features
            y_train (array): testing labels
            y_test (array): testing labels
            parameter_space (dict): hyperopt compatible parameter space
            model (module pointer): machine learning model compatiable with parameter space
            loss_function (object): function to calculate the desired loss. Note loss will be inverted in output
            loss_params (dict): additional arguments to be passed to loss function
            loss_pred_proba (bool): whether loss function uses predicted probs (True) or predicted values (False)
        """
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.model = model
        self.loss_function = loss_function
        self.loss_params = loss_params
        self.loss_pred_proba = loss_pred_proba

        if parameter_space is None:
            self.parameter_space = {
                "n_estimators": hp.choice(
                    "n_estimators", np.arange(10, dtype=int) * 30 + 50
                ),
                "criterion": hp.choice(
                    "criterion",
                    np.array(["gini", "entropy"]),
                ),
                "max_depth": hp.choice("max_depth", np.arange(21, dtype=int) + 2),
                "min_samples_split": hp.uniform("min_samples_split", 0.0, 0.05),
                "max_samples": hp.uniform("max_samples", 0.5, 0.95),
                "n_jobs": hp.choice("n_jobs", [-1]),
                "random_state": hp.choice("random_state", [1738]),
                "bootstrap": hp.choice("bootstrap", [True]),
            }
        else:
            self.parameter_space = parameter_space

    def objective(self, params):
        """Objective function for loss that is provided to perform the MINLP
        optimizaiton in hyperopt

        Args:
            params (dict): hyperopt formated dictionary of hyperparameters

        Returns:
            dict: loss and status for hyperopt optimization
        """
        model = self.model(**params)
        model.fit(self.X_train, self.y_train)
        if self.loss_pred_proba:
            pred = model.predict_proba(self.X_test)
        else:
            pred = model.predict(self.X_test)
        loss = 1 - self.loss_function(self.y_test, pred[:, 1], **self.loss_params)
        return {"loss": loss, "status": STATUS_OK}

    def optimize(self, max_evals=20):
        """optimizes the hyperparameter space in the object

        Args:
            max_evals (int): number of hyperopt iterations

        Returns:
            dict: best hyperparameters
        """
        trials = Trials()
        best = fmin(
            fn=self.objective,
            space=self.parameter_space,
            algo=tpe.suggest,
            max_evals=max_evals,
            trials=trials,
        )
        return space_eval(self.parameter_space, best)


This is the DSDE standard process for using Xboost with hyperopt

In [ ]:
#| export


def train_xgb(X_train,
              X_valid,
              y_train,
              y_valid,
              evals=20,
              sub=200000,
              train=1000000,
              early_stop=10,
              verbose=False,
              **kwargs):
    """
    Binary Classifiation Xgboost DSDE hyper opt approach this should be
    reviewed and customized for your use case.
    """
    parameter_space = {
        'max_depth': hp.choice('max_depth', np.arange(21, dtype=int) + 3),
        'n_estimators': hp.choice('n_estimators', np.arange(301, dtype=int) + 100),
        'gamma': hp.uniform('gamma', 0, 5),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 1, 0.01),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'base_score': hp.quniform('base_score', 0.3, 0.8, 0.01),
        'objective': hp.choice('objective', ['binary:logistic']),
        'eval_metric': hp.choice('eval_metric', ['logloss', 'error', 'auc', 'aucpr', 'map']),
        'use_label_encoder': hp.choice('use_label_encoder', [False]),
        'gpu_id': hp.choice('gpu_id', [-1]),
    }
    logging.info(f'Hyper tuning on {X_train[0:sub].shape[0]} rows')
    opt = HpOptBinary(
        X_train=X_train[0:sub].to_numpy(),
        X_test=X_valid.to_numpy(),
        y_train=y_train[0:sub].to_numpy(),
        y_test=y_valid.to_numpy(),
        parameter_space=parameter_space
    )
    best = opt.optimize(max_evals=evals)
    logging.info(f'Full training on {X_train[0:train].shape[0]} rows')
    model = xgb.XGBClassifier(**best, n_jobs=-1)
    eval_set = [(X_valid, y_valid)]
    model.fit(X_train[0:train],
              y_train[0:train],
              eval_set=eval_set,
              early_stopping_rounds=early_stop,
              verbose=verbose)
    return model

In [ ]:
#| export




def train_xgb_basic(X_train, X_valid, y_train, y_valid, early_stop=10, verbose=True, *args):
    """
    Binary Classifiation Xgboost Sklearn API Call Basic HyperParameters
    """
    logger.info(f'Training on {X_train.shape[0]} rows')
    model = xgb.XGBClassifier(n_jobs=-1)
    eval_set = [(X_valid, y_valid)]
    model.fit(X_train, y_train, eval_set=eval_set, early_stopping_rounds=early_stop, verbose=verbose)
    return model

In [ ]:
#| export




def train_logistic(X_train, X_test, y_train, y_test, model=LogisticRegression, evals=20, sub=200000, train=1000000):
    """Logistic Regression Example to show how simple this can be to switch the model being used in this template"""
    parameter_space = {
        'class_weight': hp.choice('class_weight', [None, 'balanced']),
        'warm_start': hp.choice('warm_start', [True, False]),
        'fit_intercept': hp.choice('fit_intercept', [True, False]),
        'tol': hp.uniform('tol', 0.00001, 0.0001),
        'C': hp.uniform('C', 0.05, 3),
        'solver': hp.choice('solver', ['lbfgs']),  # , 'liblinear', 'sag', 'saga']),
        'max_iter': hp.choice('max_iter', range(5, 1000))
    }
    logger.info(f'Hyper tuning on {X_test[0:sub].shape[0]} rows')
    opt = HpOptBinary(X_train[0:sub], X_test[0:sub], y_train[0:sub],
                      y_test[0:sub], model=model, parameter_space=parameter_space)
    best = opt.optimize(max_evals=evals)
    logger.info(f'Full training on {X_test[0:train].shape[0]} rows')
    model = LogisticRegression(**best, n_jobs=-1)
    model.fit(X_train[0:train], y_train[0:train])
    return model

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()